# 15 puzzle

<img src="img/1.jpg">

The 15 puzzle, a classic tile-sliding challenge, boasts countless variations of the "solved" state. I'm focusing on the iconic arrangement pictured above, where tiles 1 through 15 slide across a 4x4 grid, leaving one space vacant.

The rules are straightforward: move any tile adjacent to the empty space into it. In the solved state, for example, you can swap the 15 rightward or the 12 downward.

No matter how scrambled the tiles, this program will chart the shortest path back to order.

## Algorithm
In addition to the representation work that I'm about to do, the main algorithm for solving the puzzle is A* search. The reason for this is that the 15 puzzle is basically a search problem. The heuristic and all the details will be explained on each step of this notebook.

## Libraries

In [1]:
# I need the deepcopy function from copy library to avoid Python's problems with memory and copies
# The heapq library will be used in the search. Specifically I need a min-heap for the frontier of exploration
from copy import deepcopy
import heapq

## Heuristic function

The heuristic function that I use in this case is based on the Manhattan distance: given the grid (which represents the puzzle with a matrix where the empty space is represented as a zero), the function returns the sum of the Manhattan distance of each pice in the puzzle with its final position (i.e the Manhattan distance of 1 plus the Manhattan distance of 2 ...)

In [2]:
def heuristic(grid:list) -> int:
    positions = {1:[0,0], 2:[0,1], 3:[0,2], 4:[0,3], 5:[1,0], 6:[1,1], 7:[1,2], 8:[1,3], 9:[2,0], 10:[2,1], 11:[2,2], 12:[2,3], 13:[3,0], 14:[3,1], 15:[3,2], 0:[3,3]}
    result = 0
    for row in range(4):
        for column in range(4):
            value = grid[row][column]
            if value == 0: continue
            result += abs(positions[value][0] - row) + abs(positions[value][1] - column)
    return result

## Representation

To represent the states of the puzzle in nodes I created the following class

In [3]:
class Node:
    # The constructor of the class
    # The numbers of the puzzle are expected to be a list of integers
    # i.e The puzzle:
    #   [1,2,3,4]
    #   [5,6,7,8]
    #   [9,10,11,12]
    #   [13,14,15,0]
    #   is expected to be written as [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0]
    # The weight of the Node is the distance between the Node and the root of the tree
    # The sequence of the nodes is an attribute of the class where I will write the movements for solving the puzzle
    def __init__(self, numbers:list, weight:int = 0, sequence:str = ""):
        self.grid = [numbers[:4], numbers[4:8], numbers[8:12], numbers[12:]]
        self.h = heuristic(self.grid) # the value of the heuristic function
        self.w = weight
        self.f = self.h + self.w # the value of the function for A*: the heuristic plus the weight
        self.sequence = sequence
        # I will create another attribute named "empty" to know where is the empty space of the puzzle. It will be helpfull for the transition functions (to validate if a movement is possible)
        for i, num in enumerate(numbers):
            if num == 0: self.empty = [i//4, i % 4]

    # Let's create some magic methods
    def __eq__(self, other): 
        # Two nodes are equal is they have the same grid
        return self.grid == other.grid
    
    def __lt__(self, other):
        # To be able to sorted the nodes in the search frontier using the minor values of the f function
        return self.f < other.f
    
    def __str__(self):
        # If we print the node, it will print the grid
        return str(self.grid[0]) + "\n" + str(self.grid[1]) + "\n" + str(self.grid[2]) + "\n" + str(self.grid[3])
    
    # Some methods to create copies and deepcopies
    def __copy__(self):
        return Node(self.grid[0] + self.grid[1] + self.grid[2] + self.grid[3], self.h, self.w, self.sequence)
    
    def __deepcopy__(self, memo):
        row1 = deepcopy(self.grid[0], memo)
        row2 = deepcopy(self.grid[1], memo)
        row3 = deepcopy(self.grid[2], memo)
        row4 = deepcopy(self.grid[3], memo)
        weight = deepcopy(self.w, memo)
        sequence = deepcopy(self.sequence, memo)
        return Node(row1 + row2 + row3 + row4, weight, sequence)

In [4]:
# A function to create a node. Specifically the initial node of the exploration tree
def create_node() -> Node:
    # The input is a line with de integers of the puzzle.
        # Please write them following the order in which they are in the puzzle
            # i.e
            #   [1,2,3,4]
            #   [5,6,7,8]
            #   [9,10,11,12]
            #   [13,14,15,0]
            #   is expected to be written as 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 0
    numbers = list(map(int, input().strip().split()))
    # The validation of the input and the puzzle
    if len(numbers) != 16: raise ValueError 
    control = {0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15}
    for n in numbers: 
        if n not in control: raise ValueError
        control.discard(n)
        
    return Node(numbers)

## Evaluation function

The evaluation function is needed to know when the algorithm finish the search

In [5]:
def evaluate(candidate:Node) -> bool:
    # First it creates a node with the solved state
    solved = Node([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0])
    # And it compares if the given candidate is solved or not
    return candidate == solved

## Transition functions

These functions represents the movements of the puzzle. Their names indicate the piece of the puzzle to move. In other words, their names indicate their position respectively to the empty space. For example, the function "down" is for moving up the piece that is down from the empty space. Or the function "right" is for moving to the left the piece that is right from the empty space.

Their structure is the following:
- Validate if the movement is possible
- Create a new node for the new movement from the given node
- Move the piece
- Update the information of the node (the position of the empty space, the new weight, the new value of the heuristic, and the new value of the f function)

In [6]:
def down(node:Node) -> Node:
    if node.empty[0] == 3: raise IndexError
    row, column = node.empty[0], node.empty[1]
    
    result = deepcopy(node)
    result.grid[row + 1][column], result.grid[row][column] = result.grid[row][column], result.grid[row + 1][column]
    result.empty[0] += 1
    
    result.sequence += "D"
    result.w += 1
    result.h = heuristic(result.grid)
    result.f = result.w + result.h
    
    return result

In [7]:
def up(node:Node) -> Node:
    if node.empty[0] == 0: raise IndexError
    row, column = node.empty[0], node.empty[1]

    result = deepcopy(node)
    result.grid[row - 1][column], result.grid[row][column] = result.grid[row][column], result.grid[row - 1][column]
    result.empty[0] -= 1
    
    result.sequence += "U"
    result.w += 1
    result.h = heuristic(result.grid)
    result.f = result.w + result.h

    return result

In [8]:
def right(node:Node) -> Node:
    if node.empty[1] == 3: raise IndexError
    row, column = node.empty[0], node.empty[1]
    
    result = deepcopy(node)
    result.grid[row][column + 1], result.grid[row][column] = result.grid[row][column], result.grid[row][column + 1]
    result.empty[1] += 1
    
    result.sequence += "R"
    result.w += 1
    result.h = heuristic(result.grid)
    result.f = result.w + result.h

    return result

In [9]:
def left(node:Node) -> Node:
    if node.empty[1] == 0: raise IndexError
    row, column = node.empty[0], node.empty[1]
    
    result = deepcopy(node)
    result.grid[row][column - 1], result.grid[row][column] = result.grid[row][column], result.grid[row][column - 1]
    result.empty[1] -= 1
    
    result.sequence += "L"
    result.w += 1
    result.h = heuristic(result.grid)
    result.f = result.w + result.h

    return result

## A* Search

First, the creation of the first node (input or the state of th puzzle)

In [10]:
originNode = create_node()

In [12]:
# Example
print(originNode)

[9, 1, 3, 15]
[13, 8, 14, 10]
[4, 12, 2, 6]
[7, 0, 11, 5]


Finally, the search of the shortest path to the solution of the puzzle

In [13]:
# A counter for the number of explored nodes
exploredNodes = 0
# This hash table is very important. It contains the explored nodes and their minimum value of f. In consequence the algorithm will not explore a subtree with a bigger value of f.
subtreesExplored = {}

# The creation of the frontier
frontier = [originNode]
# In order to find the optimal solution, the algorithm needs to explore the nodes with the smallest value of f. To achieve this, the frontier needs to be a minheap.
heapq.heapify(frontier)

# While the frontier is not empty, the algorithm will continue the exploration
while frontier:
    # It takes out a node from the frontier (the node with the smallest value of f)
    node = heapq.heappop(frontier)

    # It evaluates if it has already explored that node with a smaller value of f
    if str(node.grid) in subtreesExplored and subtreesExplored[str(node.grid)] <= node.f: continue
    else: subtreesExplored.update({str(node.grid): node.f})

    # If the given node is the solved state, the search is completed 
    if evaluate(node): break
    # If not, it explores the node
    exploredNodes += 1

    # It applies the transition functions and if they return a node, the algorithm push it into the frontier 
    try: 
        u = deepcopy(up(node))
        heapq.heappush(frontier, u)
    except: pass
    try:
        d = deepcopy(down(node))
        heapq.heappush(frontier, d)
    except: pass
    try:
        r = deepcopy(right(node))
        heapq.heappush(frontier, r)
    except: pass
    try:
        l = deepcopy(left(node))
        heapq.heappush(frontier, l)
    except: pass

# When the search is over the output is the following
print(f"Explored nodes: {exploredNodes}")
print("Original state of the puzzle:")
print(originNode)
print("Solved state of the puzzle:")
print(node)
print("Sequence of moves:")
print(node.sequence)

Explored nodes: 542898
Original state of the puzzle:
[9, 1, 3, 15]
[13, 8, 14, 10]
[4, 12, 2, 6]
[7, 0, 11, 5]
Solved state of the puzzle:
[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 11, 12]
[13, 14, 15, 0]
Sequence of moves:
ULUURDDRURDDLURUULLDDRRUULLDDDLUURRDLDRRULLURRULDDDR


## Conclusions

With the limitations of the algorithm, given a valid state of the puzzle, its performance works well in most of the cases. Remember that the main goal of the algorithm is to find the shortest possible solution to the puzzle.

However, in some cases the exploration will be too long because the initial state is too difficult for our heuristic. So, in those cases the algorithm is limited to the memory of the computer where it is being executed. (It may take a long time to find the solution, but it will get it anyway).